In [1]:
from select import select
from odps import ODPS
from odps import options
from odps.df import DataFrame
import pandas as pd
import numpy as np
import re
import math
from collections import Counter
from collections import defaultdict

ACCESS_ID = 'LTAI5t9uwJrh5eJ7Q5E37D1s'
SECRET_ACCESS_KEY = 'NCFHOAnvqfnTrpypgR4b3cNawP8fnB'
ODPS_PROJECT = 'OpenDigger_prod_dev'
ODPS_ENDPOINT = 'http://service.cn-shanghai.maxcompute.aliyun.com/api'
# ACCESS_ID = 'LTAI5tSjrYp6JDW2bbiTdegU'
# SECRET_ACCESS_KEY = 'OE6JbSqOZUey5fzr9Wg6fuYYKvslZx'
# 创建odps实例
o = ODPS(ACCESS_ID, SECRET_ACCESS_KEY,
         project=ODPS_PROJECT, endpoint=ODPS_ENDPOINT)
options.tunnel.limit_instance_tunnel = False
# options.read_timeout = 3600000

users = DataFrame(o.get_table('ods_github_users'))
print(users.dtypes)

github_log = DataFrame(o.get_table('dim_github_actor'))
print(github_log.dtypes)

odps.Schema {
  login                     string        
  created_at                date          
  database_id               int64         
  location                  string        
  company                   string        
  bio                       string        
  is_employee               boolean       
  email                     string        
  infoname                  string        
  followers                 string        
  following                 string        
  time                      date          
  name                      string        
  lastupdatedat             date          
  nextupdateat              date          
}

odps.Schema {
  actor_id              int64           
  actor_login           string          
  type                  string          
  created_at            datetime        
}



In [2]:
def exe_sql(sql):
    data = []
    with o.execute_sql(sql).open_reader() as reader:
        d = defaultdict(list)  
        for record in reader:
            for res in record:
                d[res[0]].append(res[1])  
        data = pd.DataFrame.from_dict(d,orient='index').T 
    return data

rdata =  exe_sql("SELECT * FROM ods_github_users;")


In [3]:
data = rdata.dropna(axis='index', how='all', subset=['followers'])
data = data.drop_duplicates(['database_id'])  
data

,login,created_at,database_id,location,company,bio,is_employee,email,infoname,followers,following,time,name,lastupdatedat,nextupdateat
0,LombiqBot,2014-08-21,8517910,"Budapest, Hungary",Lombiq Technologies Ltd.,I'm a friendly robot that can also pass the Tu...,False,bot@lombiq.com,Lombiq Bot,"50567111,61306948,60854565,57724193,55095689,5...",None,2021-11-22,LombiqBot,2022-01-25,2022-02-24
3,direwolf-github,2015-02-02,10810283,None,None,None,False,,None,"57059290,35817050,56658371,45868471",None,2021-11-18,direwolf-github,2022-01-20,2022-02-19
9,renovate-bot,2017-01-17,25180681,None,@whitesource,WhiteSource Renovate is a bot to keep dependen...,False,renovate@whitesourcesoftware.com,WhiteSource Renovate,"23411452,93843436,1330362,82955280,69742740,40...",None,2021-11-09,renovate-bot,2022-01-11,2022-02-10
12,breakingheatmap,2020-07-17,68417258,None,None,None,False,,None,"9112378,17136125,1753262,53253777",None,2021-11-28,breakingheatmap,2022-01-29,2022-02-28
15,codestar-github-bot-1,2019-07-20,53108186,None,None,None,False,,None,51524117,None,2021-11-24,codestar-github-bot-1,2021-12-28,2022-01-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618234,bored-cpu,2011-06-12,845717,Metaverse,None,,False,,None,"13079840,912821,34725630,829214,418638,5039239...","1996030,13079840,8121224,24038,4890546,8460,40...",2022-01-29,bored-cpu,2022-01-29,2022-02-28
618235,CaerwynBartley,2015-09-17,14330070,Australia,None,Making cool stuff and seeing where it takes me,False,c_barts@live.com,Caerwyn Bartley,"32235163,43767793","33310669,71807496,71798673,32116730,70354104",2022-01-31,CaerwynBartley,2022-01-31,2022-03-02
618238,denllx,2018-08-14,42374096,None,None,,False,,Higari,"93073000,38338565,18473413,48854566,32595111,2...","31069308,22500116,38338565,18473413,48854566,3...",2022-01-30,denllx,2022-01-30,2022-03-01
618239,bkiscool,2017-11-23,33919755,None,None,,False,,bk,79547580,"79547580,6016498",2022-02-02,bkiscool,2022-02-02,2022-03-04


In [4]:
import networkx as nx
G = nx.DiGraph()

In [5]:
for index, row in data.iterrows():
    follower_list = row['followers'].split(',')
    for follower in follower_list:
         G.add_edge(follower, row['database_id'])


In [12]:
nx.write_edgelist(G,"follow_net.txt",data = False)

In [ ]:
G = nx.read_edgelist('follow_net.txt')

In [11]:
G.number_of_edges()

11739301

In [6]:
pr = nx.pagerank(G, alpha=0.85)

In [7]:
c=Counter(pr).most_common() 

In [8]:
c[0]

(1024025, 0.015270519416542432)

In [9]:
with open('pagerank.txt', 'w') as f:
    for i in range(len(c)):
        f.write(str(c[i]) + '\r')
f.close()

In [14]:
d = rdata.loc[rdata['database_id'] == 1024025]
d

,login,created_at,database_id,location,company,bio,is_employee,email,infoname,followers,following,time,name,lastupdatedat,nextupdateat
82236,torvalds,2011-09-03,1024025,"Portland, OR",Linux Foundation,None,False,,Linus Torvalds,"49609699,45284106,59277828,16850222,45160807,6...",None,2021-11-20,torvalds,2022-01-02,2022-02-01
82237,torvalds,2011-09-03,1024025,"Portland, OR",Linux Foundation,None,False,,Linus Torvalds,"1650022,61854273,9489269,29665300,49203535,172...",None,2022-01-02,torvalds,2022-01-02,2022-02-01


In [15]:
d = rdata.loc[rdata['database_id'] == 66577]
d

,login,created_at,database_id,location,company,bio,is_employee,email,infoname,followers,following,time,name,lastupdatedat,nextupdateat
23266,JakeWharton,2009-03-24,66577,"Pittsburgh, PA, USA",@square / @cashapp,,False,j@ke.fyi,Jake Wharton,"4854092,77495381,16327829,33611796,82707090,16...","46553,1675456,3945493,227486,133019,170788,869...",2021-11-14,JakeWharton,2022-01-22,2022-02-21
23267,JakeWharton,2009-03-24,66577,"Pittsburgh, PA, USA",@square / @cashapp,,False,j@ke.fyi,Jake Wharton,"64865577,91460378,43780800,20805493,26974575,1...","46553,1675456,3945493,227486,133019,170788,869...",2021-12-19,JakeWharton,2022-01-22,2022-02-21
23268,JakeWharton,2009-03-24,66577,"Pittsburgh, PA, USA",@square / @cashapp,,False,j@ke.fyi,Jake Wharton,"85793956,57993597,59120057,27435736,330693,702...","46553,1675456,3945493,227486,133019,170788,869...",2022-01-22,JakeWharton,2022-01-22,2022-02-21


In [16]:
d = rdata.loc[rdata['database_id'] == 905434]
d

,login,created_at,database_id,location,company,bio,is_employee,email,infoname,followers,following,time,name,lastupdatedat,nextupdateat
555657,ruanyf,2011-07-10,905434,"Shanghai, China",None,None,False,yifeng.ruan@gmail.com,Ruan YiFeng,"25890579,92638493,64113937,46164858,34157144,7...",None,2021-11-13,ruanyf,2022-01-21,2022-02-20
555658,ruanyf,2011-07-10,905434,"Shanghai, China",None,None,False,yifeng.ruan@gmail.com,Ruan YiFeng,"56532671,29500912,89166015,96288219,22735326,7...",None,2021-12-17,ruanyf,2022-01-21,2022-02-20
555659,ruanyf,2011-07-10,905434,"Shanghai, China",None,None,False,yifeng.ruan@gmail.com,Ruan YiFeng,"49703877,96920091,77330637,96614937,90494741,9...",None,2022-01-21,ruanyf,2022-01-21,2022-02-20


In [17]:
d = rdata.loc[rdata['database_id'] == 499550]
d

,login,created_at,database_id,location,company,bio,is_employee,email,infoname,followers,following,time,name,lastupdatedat,nextupdateat
10469,yyx990803,2010-11-28,499550,New Jersey ⇄ China,vuejs,,False,,Evan You,"94733895,20681063,50052008,74041428,13210466,8...","9126398,7929704,6685088,4474660,19343,747051,8...",2021-11-22,yyx990803,2022-01-24,2022-02-23
10470,yyx990803,2010-11-28,499550,Singapore,vuejs,,False,,Evan You,"94895661,6166802,60646455,47776374,42918239,76...","9126398,7929704,6685088,4474660,19343,747051,8...",2021-12-25,yyx990803,2022-01-24,2022-02-23
10471,yyx990803,2010-11-28,499550,Singapore,vuejs,,False,,Evan You,"64653332,73787295,56951960,62935381,59045716,5...","9126398,7929704,6685088,4474660,19343,747051,8...",2022-01-24,yyx990803,2022-01-24,2022-02-23


In [ ]:
hub, authority = nx.hits(G)

In [ ]:
h=Counter(hub).most_common() 
a=Counter(authority).most_common() 

In [ ]:
with open('hits_h.txt', 'w') as f:
    length = len(h)
    for i in range(length):
        f.write(str(hub[i]) + '\r')
f.close()

with open('hits_a.txt', 'w') as f:
    length = len(a)
    for i in range(length):
        f.write(str(authority[i]) + '\r')
f.close()